In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [28]:
response_obj = requests.get('https://en.wikipedia.org/wiki/Special_wards_of_Tokyo').text
soup = BeautifulSoup(response_obj,'lxml')
Wards_Tokyo_Table = soup.find('table', {'class':'wikitable sortable'})
### preparation of the table 

Name=[]
Kanji = []
Pop = []
Density = []
num = []
flag = []
Area = []
Major_District = []

for row in Wards_Tokyo_Table.findAll("tr"):
    #print (row)    
    Ward = row.findAll('td')
    #print (len(Ward))
    print (Ward)
    if len(Ward)==8: #Only extract table body not heading
        print (Ward[0])
        #print postcode[1]
        #print postcode[2]
        num.append(Ward[0].find(text=True))
        flag.append(Ward[1].findAll('a')) # useless
        Name.append(Ward[2])
        Kanji.append(Ward[3].find(text=True))
        Pop.append(Ward[4])
        Area.append(Ward[5].find(text=True))
        Major_District.append(Ward[7].find(text=True))
            
#print (Pop) 


#++++++++++++++++++++++++++++++++++++++++++++++
#+ Area 
#++++++++++++++++++++++++++++++++++++++++++++++

# print (Area[3]) # the first element of the area needs a replacement with the true value 5100
Area = ['5100' if x=='0' else x for x in Area]
New_Area = []

# change the type of Area list 
for l in range(len(Area)):
    x=Area[l].replace(",","")
    #print (x)
    New_Area.append(x)

New_Area=[int(s) for s in New_Area]

#print (New_Area) # the list elements are already in accordance with the table

#+++++++++++++++++++++++++++++++++++++++++++++++++++++
#+ Name of the Wards
#+++++++++++++++++++++++++++++++++++++++++++++++++++++
#print (Name) # want to select only the title part

new_names = []
for n in range(len(Name)):
    print (Name[n])
    names = Name[n].findAll('a')
    new_names.append(names) 

#print (new_names)

flat_new_names_list = [item for sublist in new_names for item in sublist]
#print (flat_new_names_list)

Wards_names= []
#now 
for name_wards in flat_new_names_list:
        Wards_names.append(name_wards.get('title'))

#print (Wards_names)

# replace the elements in the list that contains 'Tokyo' with only the ward names
replace_names={'Chiyoda, Tokyo':'Chiyoda', 'Chūō, Tokyo':'Chuo', 'Minato, Tokyo':'Minato', 
               'Sumida, Tokyo':'Sumida', 'Koto, Tokyo':'Koto', 'Ōta, Tokyo':'Ota', 'Nakano, Tokyo':'Nakano', 
               'Kita, Tokyo':'Kita', 'Arakawa, Tokyo':'Arakawa', 'Adachi, Tokyo':'Adachi', 'Edogawa, Tokyo':'Edogawa',
               'Taitō':'Taito','Bunkyō':'Bunkyo'}


Wards_names1 = [replace_names.get(n1,n1) for n1 in Wards_names]

#print (Wards_names1)

#+++++++++++++++++++++++++++++++++++++++++++++++++++++
#+ Population
#+++++++++++++++++++++++++++++++++++++++++++++++++++++
# print (len(Pop))
# #print ((Pop[5].text))
population = []
for p in range(len(Pop)):
    print ((Pop[p]))
    pops = Pop[p].text[1:9]
    print (Pop[p].text[1:9])
    #populs = Pop[p].find('visibility:hidden;color:transparent;')
    population.append(pops) 
#print (population)


New_population = []
for po in range(len(population)):
    xy=population[po].replace(",","")
    print (xy)
    New_population.append(xy)

New_population=[int(s1) for s1 in New_population]
# print (New_population)



#++++++++++++++++++++++++++++++++++++++++++++++++
#+ Major Districts
#++++++++++++++++++++++++++++++++++++++++++++++++


replace_districts = {'Nagatachō':'Nagatacho', 'Hongō':'Hongo', 'Kinshichō':'Kinshicho', 'Ōmori': 'Omori', 
                     'Kōenji':'Koenji', 'Arakawa, Machiya, ':'Arakawa', 'Ayase, ':'Ayase', 'Kasai, Koiwa\n':'Kasai'}


Major_District_names1 = [replace_districts.get(n2,n2) for n2 in Major_District]


df=pd.DataFrame(Wards_names1,columns=['Ward'])
df['Area_SqKm'] = New_Area
df['Population'] = New_population
df['Major_District'] = Major_District_names1
df.index = np.arange(1, len(df) + 1) # reset the index so that it starts from 1. 

# use of geopy geocoders
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Tokyo_explorer")
df['Major_Dist_Coord']= df['Major_District'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df[['Latitude', 'Longitude']] = df['Major_Dist_Coord'].apply(pd.Series)

df.drop(['Major_Dist_Coord'], axis=1, inplace=True)
# replace some of the wrong coordinates 


Lat_list = df['Latitude'].tolist()
Long_list = df['Longitude'].tolist()
print ("Old latitude list: ", Lat_list)
print ("Old Longitude list: ", Long_list)
replace_latitudes = {32.5093796:35.7088, 23.0131338:35.6722, -38.9047057:35.5884, -5.3498001:35.6634}
replace_longitudes = {-116.2970014:139.7601, -80.8328748:139.8061, 175.7552111:139.7279, 21.424098:139.8731}

latitudes_new = [replace_latitudes.get(n3,n3) for n3 in Lat_list]
longtitudes_new = [replace_longitudes.get(n4,n4) for n4 in Long_list]
print (latitudes_new)
print (longtitudes_new)

Tokyo_df = df.drop(['Latitude', 'Longitude'], axis=1)
# #df.drop(['Longitude'], axis=1, inplace=True)
# Tokyo_df


Tokyo_df['Dist_Latitude'] = latitudes_new
Tokyo_df['Dist_Longitude'] = longtitudes_new

#Tokyo_df

Tokyo_df.to_csv('Tokyo_df_Coord.csv')

Old latitude list:  [35.675618, 35.68406775, 35.619050200000004, 35.6937632, 35.1753761, 35.7133757, 35.6963122, 23.0131338, 35.599252, 35.62125, 35.5884735, 35.6616779, 35.6645956, 35.718123, 35.7049419, 35.7310839, 35.7781394, 35.737529, 35.774143, 35.74836, 35.4460475, 33.4817913, -5.3498001]
Old Longitude list:  [139.7434685, 139.77450291683806, 139.77936404588297, 139.7036319, 137.0134764, 139.7766557, 139.8150431, -80.8328748, 139.73891, 139.688014, 139.7279334, 139.6663345, 139.6987107, 139.664468, 139.649909, 139.7089164, 139.7207999, 139.78131, 139.681209, 139.638735, 139.430823, 131.4781536, 21.424098]
[35.675618, 35.68406775, 35.619050200000004, 35.6937632, 35.1753761, 35.7133757, 35.6963122, 35.6722, 35.599252, 35.62125, 35.5884735, 35.6616779, 35.6645956, 35.718123, 35.7049419, 35.7310839, 35.7781394, 35.737529, 35.774143, 35.74836, 35.4460475, 33.4817913, 35.6634]
[139.7434685, 139.77450291683806, 139.77936404588297, 139.7036319, 137.0134764, 139.7766557, 139.8150431, 139

In [29]:
Tokyo_df = pd.read_csv('Tokyo_df_Coord.csv', index_col=[0])
#remove the unnamed column 
Tokyo_df

,Ward,Area_SqKm,Population,Major_District,Dist_Latitude,Dist_Longitude
1,Chiyoda,5100,59441,Nagatacho,35.675618,139.743469
2,Chuo,14460,147620,Nihonbashi,35.684068,139.774503
3,Minato,12180,248071,Odaiba,35.619050,139.779364
4,Shinjuku,18620,339211,Shinjuku,35.693763,139.703632
5,Bunkyo,19790,223389,Hongo,35.175376,137.013476
6,Taito,19830,200486,Ueno,35.713376,139.776656
7,Sumida,18910,260358,Kinshicho,35.696312,139.815043
8,Koto,12510,502579,Kiba,35.672200,139.806100
9,Shinagawa,17180,392492,Shinagawa,35.599252,139.738910
10,Meguro,19110,280283,Meguro,35.621250,139.688014


<h1> Get the name and location of Tokyo Hospitals 

In [4]:
allianz = requests.get("https://apps.allianzworldwidecare.com/poi/hospital-doctor-and-health-practitioner-finder?PROVTYPE=HOSPITALS&CON=Asia&COUNTRY=Japan&CITY=Tokyo&choice=en").text

In [3]:
soup_hospital =  BeautifulSoup(allianz,"html.parser")
hospital_info_table = soup_hospital.find('div',{'class':"table-responsive"})

In [ ]:
print(hospital_info_table)

In [5]:
hospital_name = []
hospital_address = []
hospital = []
for row in hospital_info_table.findAll('table'):
    hospital = row.findAll('th') #Name of hospital
    for name in hospital:
        hospital_name.append(name.text)
        
        hospital2 = row.select('span')
        for address in hospital2:    #Address of hospitals
               hospital_address.append(address.text.replace(',',''))
    

hospital_address[0:5] = [''.join(hospital_address[0:5])]
hospital_address[1:6] = [''.join(hospital_address[1:6])]
hospital_address[2:6] = [''.join(hospital_address[2:6])]
hospital_address[3:8] = [''.join(hospital_address[3:8])]
hospital_address[4:8] = [''.join(hospital_address[4:8])]
hospital_address[5:10] = [''.join(hospital_address[5:10])]
hospital_address[6:11] = [''.join(hospital_address[6:11])]
hospital_address[7:12] = [''.join(hospital_address[7:12])]
hospital_address[8:13] = [''.join(hospital_address[8:13])]
hospital_address[9:13] = [''.join(hospital_address[9:13])]
hospital_address[10:13] = [''.join(hospital_address[10:13])]
hospital_address[11:16] = [''.join(hospital_address[11:16])]
hospital_address[12:15] = [''.join(hospital_address[12:15])]
hospital_address[13:17] = [''.join(hospital_address[13:17])]
hospital_address[14:17] = [''.join(hospital_address[14:17])]
hospital_address[15:20] = [''.join(hospital_address[15:20])]

print(hospital_address)
print(len(hospital_name))
print(len(hospital_address))
#print(list(set(hospital_address)))

['1-16-10 Shibaura Minato-ku Tokyo 105-8321 Japan.', '2-41-2 NTT 1. Floor Denenchofu Ota-Ku Tokyo 145-0071 Japan.', '5-1 Naka Ochiai 2-chome Shinjuku-ku Tokyo Japan.', '3-19-18 Nishi Shinbashi Minato Tokyo 105-0003 Japan.', '6-20-2ShinkawaMitaka Tokyo 1818611 Japan.', '1-12-15 Sendagi Bunkyo-ku Tokyo 113-0022 Japan.', '8-10-16 Akasaka Minato-Ku Tokyo 107-0052 Japan.', '9-1 Akashico Chuo-ku Tokyo 104-8560 Japan.', '7-3-1 Hongo Bunkyo Tokyo 113-0033 Japan.', '3-17-3 Amanuma Suginami-Ku Tokyo 67-8507 Japan.', '19-3 Nishigotanda 2 Chome Shinagawaku Tokyo Japan.', '32 Shiba Koen Building 2F 3-4-3 Shiba Koen Minato-ku Tokyo Japan.', 'Midtown Tower 6F Akasaka 9-7-1 Minato-ku Tokyo Japan.', '3-1-1 Matsubara-cho Akishima-shi Tokyo 196-0003 Japan.', '1-4-17 Wita Minatoku Tokyo Japan.', '2-3-8-1F Azabudai Minato-ku Tokyo 106-0041 Japan.']
16
16


In [8]:
#Append hospital name and hospital address into one table
hospital_df = pd.DataFrame(hospital_name,columns = ["Hospital"])
hospital_df['Address'] = hospital_address
hospital_df

,Hospital,Address
0,Aiiku Hospital,1-16-10 Shibaura Minato-ku Tokyo 105-8321 Japan.
1,Denen Chofu Osada Orthopaedic Clinic,2-41-2 NTT 1. Floor Denenchofu Ota-Ku Tokyo 14...
2,International Catholic Hospital (Seibo Byoin),5-1 Naka Ochiai 2-chome Shinjuku-ku Tokyo Japan.
3,Jikei University Hospital,3-19-18 Nishi Shinbashi Minato Tokyo 105-0003 ...
4,Kyorin University Hospital,6-20-2ShinkawaMitaka Tokyo 1818611 Japan.
5,Nippon Medical School Hospital,1-12-15 Sendagi Bunkyo-ku Tokyo 113-0022 Japan.
6,Sanno Hospital,8-10-16 Akasaka Minato-Ku Tokyo 107-0052 Japan.
7,St. Luke's International Hospital,9-1 Akashico Chuo-ku Tokyo 104-8560 Japan.
8,The University of Tokyo Hospital,7-3-1 Hongo Bunkyo Tokyo 113-0033 Japan.
9,Tokyo Adventsist Hospital,3-17-3 Amanuma Suginami-Ku Tokyo 67-8507 Japan.


In [20]:
#Separate wards name within address
whitelist = set(['Minato-ku','Minato-Ku','Ota-Ku','Shinjuku-ku','Minato','Mitaka','Bunkyo-ku','Bunkyo','Chou-ku','Suginami-Ku','Shinagawaku'
            ,'Minatoku'])
''.join([wards for wards in hospital_address if wards in whitelist])


''

In [26]:
value = '123456'
whitelist = set('25')
''.join([c for c in value if c in whitelist])

'25'

<h1> Get the number of cases per special ward areas

In [31]:
tokyo_cases = pd.read_csv('tokyo_cases_byarea.csv')
tokyo_patients = pd.read_csv('tokyo_covid19_patients.csv')
#Get special wards areas only
tokyo_cases = tokyo_cases.iloc[0:23]
tokyo_cases = tokyo_cases.drop(['Code','Area'],axis = 1)
tokyo_cases = tokyo_cases.rename(columns = {"Municipality":"Ward"})
tokyo_cases.head(5)

,Ward,Positive Cases
0,Adachi,4769
1,Arakawa,1591
2,Itabashi,3683
3,Edogawa,3853
4,Ota,5255


In [32]:
#Merge population table with cases-per-ward table
Tokyo_df_edited = pd.merge(tokyo_cases,Tokyo_df,how="left",on="Ward")
Tokyo_df_edited.head(5)

,Ward,Positive Cases,Area_SqKm,Population,Major_District,Dist_Latitude,Dist_Longitude
0,Adachi,4769,12660,674067,Ayase,35.446047,139.430823
1,Arakawa,1591,21030,213648,Arakawa,35.737529,139.781310
2,Itabashi,3683,17670,569225,Itabashi,35.774143,139.681209
3,Edogawa,3853,13750,685899,Kasai,35.663400,139.873100
4,Ota,5255,11910,722608,Omori,35.588473,139.727933


In [23]:
tokyo_patients

,No,Date,Date (Onset),Region,Age,Gender,Situation
0,1,2020/1/24,NaN,"Wuhan, China",40's,Male,Discharged
1,2,2020/1/25,NaN,"Wuhan, China",30's,Female,Discharged
2,3,2020/1/30,NaN,"Changsha, China",30's,Female,Discharged
3,4,2020/2/13,NaN,Tokyo,70's,Male,Discharged
4,5,2020/2/14,NaN,Tokyo,50's,Female,Discharged
...,...,...,...,...,...,...,...
100785,100740,2021/2/2,NaN,NaN,80's,Female,Not Discharged
100786,100741,2021/2/2,NaN,NaN,10's,Female,Not Discharged
100787,100742,2021/2/2,NaN,NaN,30's,Male,Not Discharged
100788,100743,2021/2/2,NaN,NaN,60's,Male,Not Discharged


<h1> Get info of nearby residential areas </h1>

In [ ]:
#!conda install -c conda-forge geocoder --yes
from geopy.geocoders import Nominatim 
import geocoder

# initialize your variable to None
lat_lng_coords = None
# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Tokyo, Japan'.format(postal_code))
  lat_lng_coords = g.latlng
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [ ]:
print(latitudes_new)
print(longtitudes_new)

In [20]:
CLIENT_ID = 'XCN05TZAWXBX4DZKB1WJTCYAK55NDPTEN3KVGNLTXSKUKGRW' # your Foursquare ID
CLIENT_SECRET = '2WJCTR3BRA2KMPGWEIBSCSSBG2ALG0L5IAWFHMZCHZ0PIVAE' # your Foursquare Secret
VERSION = '20180605'

In [24]:
def getVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
tokyo_venues = getVenues(names=Tokyo_df['Ward'],
                                   latitudes=Tokyo_df['Dist_Latitude'],
                                   longitudes=Tokyo_df['Dist_Longitude']
                                  )

Chiyoda
Chuo
Minato
Shinjuku
Bunkyō
Taitō
Sumida
Koto
Shinagawa
Meguro
Ota
Setagaya
Shibuya
Nakano
Suginami
Toshima
Kita
Arakawa
Itabashi
Nerima
Adachi
Katsushika
Edogawa


In [30]:
tokyo_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chiyoda,35.675618,139.743469,Nagatacho Kurosawa (永田町 黒澤),35.674699,139.741737,Japanese Restaurant
1,Chiyoda,35.675618,139.743469,The Capitol Hotel Tokyu (ザ・キャピトルホテル東急),35.673927,139.741019,Hotel
2,Chiyoda,35.675618,139.743469,Tully's Coffee,35.674594,139.743007,Coffee Shop
3,Chiyoda,35.675618,139.743469,Shinamen Hashigo (支那麺 はしご),35.672184,139.741576,Ramen Restaurant
4,Chiyoda,35.675618,139.743469,HOSHIGAOKA / STAR HILL (星ヶ岡),35.673953,139.740998,Chinese Restaurant
...,...,...,...,...,...,...,...
1386,Edogawa,35.663400,139.873100,chautari チョウタリ,35.659341,139.872934,Indian Restaurant
1387,Edogawa,35.663400,139.873100,ササキスポーツ,35.661123,139.868824,Sporting Goods Shop
1388,Edogawa,35.663400,139.873100,長島町交差点,35.667695,139.873194,Intersection
1389,Edogawa,35.663400,139.873100,中葛西八丁目交差点,35.658935,139.872812,Intersection


In [31]:
tokyo_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adachi,4,4,4,4,4,4
Arakawa,33,33,33,33,33,33
Bunkyō,24,24,24,24,24,24
Chiyoda,33,33,33,33,33,33
Chuo,100,100,100,100,100,100
Edogawa,84,84,84,84,84,84
Itabashi,19,19,19,19,19,19
Katsushika,3,3,3,3,3,3
Kita,100,100,100,100,100,100


In [32]:
tokyo_venues.count()

Neighbourhood              1391
Neighbourhood Latitude     1391
Neighbourhood Longitude    1391
Venue                      1391
Venue Latitude             1391
Venue Longitude            1391
Venue Category             1391
dtype: int64

In [33]:
# Make a folium map illustrating areas with most cases with KNN clusters 